In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
%cd /content/drive/MyDrive/711_as3/dataAug/

In [4]:
import pandas as pd
import numpy as np

train_csv_path = 'train.tsv'
original_train = pd.read_csv(train_csv_path, sep='\t', header=None)

In [ ]:
print('train shape:', original_train.shape)
original_train.head(10)

In [6]:
X_train = original_train[0]
y_train = original_train[1]


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train)
print(y_train)

### Augmentation using BERT embeddings

In [ ]:
! pip install nlpaug==1.1.7

In [11]:
# apply augmentation to the train data and save the results into a file
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action
import nlpaug.flow as naf

aug_bert = naf.Sequential([naf.Sometimes([
    naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert", device ='cuda', batch_size=64)]),
    naf.Sometimes([naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", device ='cuda', batch_size=64)
])])

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

data = {'text': X_train, 'label': y_train}
base_train = pd.DataFrame.from_dict(data)
base_train.shape

In [ ]:
rep = 5
bert_train = base_train.copy()
bert_train['paraphrase'] = bert_train['text'].progress_apply(lambda x:aug_bert.augment(x, rep))
bert_train = bert_train.explode('paraphrase').reset_index(drop=True)
bert_train

In [16]:
# drop the text col
augmented_train = bert_train.drop(columns=['text'])

columns_titles = ["paraphrase", "label"]
augmented_train = augmented_train.reindex(columns=columns_titles)

In [ ]:
# concat augmented and original
base_train = base_train.rename(columns={'text': 'paraphrase'})
final_train = pd.concat([augmented_train, base_train], axis=0, ignore_index=True)
final_train.shape

In [25]:
# save final augmented dataset
final_train.to_csv('train_bertEmbed_augmented.tsv', sep="\t", encoding='utf-8', index=False, header=None)